# Анализ взаимодействия пользователей с карточками статей Яндекс.Дзен #

## Введение ##

**Цель:** 

автоматизировать анализ взаимодействия пользователей с карточками статей Яндекс.Дзен.

**Задача:** 
    
сделать дашборд, который будет основываться на пайплайне, который будет брать данные из таблицы, в которых хранятся сырые данные, трансформировать данные и укладывать их в агрегирующую таблицу. Пайплайн будет разработан дата-инженерами.


## Техническое задание: ##

**Бизнес-задача:** анализ взаимодействия пользователей с карточками Яндекс.Дзен;

**Насколько часто предполагается пользоваться дашбордом:** не реже, чем раз в неделю;

**Кто будет основным пользователем дашборда:** менеджеры по анализу контента;

**Состав данных для дашборда:**

- История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
- Разбивка событий по темам источников;
- Таблица соответствия тем источников темам карточек;

**По каким параметрам данные должны группироваться:**

- Дата и время;
- Тема карточки;
- Тема источника;
- Возрастная группа;

**Характер данных:**

- История событий по темам карточек - абсолютные величины с разбивкой по минутам;
- Разбивка событий по темам источников - относительные величины (% событий);
- Соответствия тем источников темам карточек - абсолютные величины;

**Важность:** все графики имеют равную важность;

**Источники данных для дашборда:** cырые данные о событиях взаимодействия пользователей с карточками (таблица log_raw);

**База данных, в которой будут храниться агрегированные данные:** дополнительные агрегированные таблицы в БД zen;

**Частота обновления данных:** один раз в сутки, в полночь по UTC;

**Макет дашборда:**

![макет](https://www.dropbox.com/scl/fi/75u7z919bnogguptvoe0a/.jpg.jpg?rlkey=2h4zycr4d11b1vr31tnvn8brb&dl=0)

## Импорт библиотек Python ##

In [16]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных
            
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Выгрузка данных ##

Запускаем скрипт, который использует запрос к базе данных для того, чтобы получить все записи из таблицы:

In [34]:
 query = '''
           SELECT *
           FROM dash_visits
        '''
sql_data = pd.io.sql.read_sql(query, con = engine)

Экспорт в csv:

In [35]:
sql_data.to_csv('dash_visits.csv', index = False)

Чтение данных из полученного файла csv:

In [36]:
visits_data = pd.read_csv('dash_visits.csv', parse_dates=['dt'])

Изучим полученные данные:

In [37]:
display(visits_data.head())
visits_data.info()

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


Преобразуем идентификатор в тип данных object:

In [38]:
visits_data['record_id'] = visits_data['record_id'].astype('object')

Изучим статистические характеристики:

In [39]:
visits_data.describe(include='all').fillna('')

,record_id,item_topic,source_topic,age_segment,dt,visits
count,30745.0,30745,30745,30745,30745,30745.0
unique,30745.0,25,26,6,17,
top,1040597.0,Отношения,Семейные отношения,18-25,2019-09-24 18:58:00,
freq,1.0,1536,1822,7056,3383,
first,,,,,2019-09-24 18:28:00,
last,,,,,2019-09-24 19:00:00,
mean,,,,,,10.089673
std,,,,,,19.727601
min,,,,,,1.0
25%,,,,,,1.0


## Вывод ##

Выгруженные данные дополнительной обработки не требуют.